In [2]:
from git import Repo
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import Chroma
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain

In [4]:
%pwd

'e:\\Study\\COMPLETED PROJECTS\\End-to-End-Source-Code-Analysis-Gen-Ai\\research'

In [3]:
!mkdir test_repo

In [5]:
repo_path = "test_repo/"

repo = Repo.clone_from("https://github.com/paravsyal02/End-to-End-Medical-Chatbot.git", to_path=repo_path)

In [6]:
loader = GenericLoader.from_filesystem(repo_path, 
                                       glob = "**/*",
                                        suffixes=[".py"],
                                        parser = LanguageParser(language=Language.PYTHON, parser_threshold=500)
)               

In [7]:
documents = loader.load()

In [8]:
documents

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_google_genai import ChatGoogleGenerativeAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)\n\n# Load environment variables\nload_dotenv()\nPINECONE_API_KEY = os.getenv("PINECONE_API_KEY")\nGOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")\n\nembeddings = download_hugging_face_embeddings()\n\nindex_name = "medicalbot"\n\n# Embed each chunk and upsert the embeddings into your Pinecone Index\ndocsearch = PineconeVectorStore.from_existing_index(\n    index_name=index_name,\n    embedding

In [9]:
len(documents)

8

In [ ]:
documents_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.PYTHON,
                                                                  chunk_size = 500,
                                                                  chunk_overlap = 20)

In [12]:
texts = documents_splitter.split_documents(documents)

In [13]:
texts

[Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='from flask import Flask, render_template, jsonify, request\nfrom src.helper import download_hugging_face_embeddings\nfrom langchain_pinecone import PineconeVectorStore\nfrom langchain_google_genai import ChatGoogleGenerativeAI\nfrom langchain.chains import create_retrieval_chain\nfrom langchain.chains.combine_documents import create_stuff_documents_chain\nfrom langchain.prompts import ChatPromptTemplate\nfrom dotenv import load_dotenv\nfrom src.prompt import *\nimport os\n\napp = Flask(__name__)'),
 Document(metadata={'source': 'test_repo\\app.py', 'language': <Language.PYTHON: 'python'>}, page_content='# Load environment variables\nload_dotenv()\nPINECONE_API_KEY = os.getenv("PINECONE_API_KEY")\nGOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")\n\nembeddings = download_hugging_face_embeddings()\n\nindex_name = "medicalbot"\n\n# Embed each chunk and upsert the embeddings into your Pine

In [14]:
len(texts)

21

In [15]:
from dotenv import load_dotenv
load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

In [17]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [18]:
vectordb = Chroma.from_documents(texts, embedding=embeddings, persist_directory='db')

In [19]:
vectordb.persist()

C:\Users\DELL\AppData\Local\Temp\ipykernel_3040\3711397106.py:1: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [20]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [28]:
memory = ConversationBufferMemory( memory_key="chat_history", return_messages=True)

In [33]:
qa = ConversationalRetrievalChain.from_llm(llm, retriever=vectordb.as_retriever(search_type="mmr", search_kwags ={"k":8}, memory=memory ))

In [ ]:
question = "What is download_hugging_face_embeddings function"

In [38]:
result = qa({"question": question, "chat_history": []})  # Ensure chat_history is included
print(result['answer'])


The `load_pdf_file` function loads all PDF files from a specified directory. It uses a `DirectoryLoader` with the `PyPDFLoader` to handle PDF parsing.  It returns a list of `Document` objects, each representing a loaded PDF file.


In [36]:
question = "What is load_pdf_file function"

In [37]:
result = qa({"question": question, "chat_history": []})  # Ensure chat_history is included
print(result['answer'])


The `load_pdf_file` function loads all PDF files from a specified directory. It uses the `DirectoryLoader` from `langchain_community.document_loaders`, specifying `PyPDFLoader` to handle PDF files. The function returns a list of `Document` objects, each representing a loaded PDF.
